# Pymaceuticals Inc.
---

### Analysis

- The drug Capomulin appears to be one of the drugs in this trial study where SCC tumors ended up being the smallest in the group next to those mice that were treated with Ramicane. There were also more timepoints observed, indicating that the mice being treated with Capomulin survived the whole trial compared to the other drugs that were being tested.

- The test group being treated with Capomulin had a smaller variance compared to its counterpart test groups, indicating that the drug's results may be more consistent than the other drugs of interest, specifically Infubinol, and Ceftamin. They both had a higher variance among the observed final tumor volumes of the mice being treated with ther respective drugs.

- One subject being treated with Capomulin, mouse i557, was observed to have its tumor decrease in size toward the beginning of the trial. However, as the trial progressed, the tumor did continue to increase in volume. A line chart was only created for this one mouse and to see if this is a trend, it is necessary to create more line charts to reveal it.

 

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single DataFrame
mouse_results = pd.merge(mouse_metadata, study_results, on="Mouse ID")
mouse_results.head()

# Display the data table for preview


In [ ]:
# Checking the number of mice.
len(mouse_results["Mouse ID"].unique())

In [ ]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
duplicates = mouse_results.loc[mouse_results.duplicated(subset=["Mouse ID", "Timepoint"]) == True, :]
duplicates["Mouse ID"]

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicate_mouse = mouse_results.loc[mouse_results["Mouse ID"] == "g989", :]
duplicate_mouse

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
remove_g989 = mouse_results.loc[mouse_results["Mouse ID"] != 'g989']
remove_g989.head()

In [ ]:
# Checking the number of mice in the clean DataFrame.
len(remove_g989["Mouse ID"].unique())

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 

mean = remove_g989.groupby("Drug Regimen")["Tumor Volume (mm3)"].mean()
median = remove_g989.groupby("Drug Regimen")["Tumor Volume (mm3)"].median()
variance = remove_g989.groupby("Drug Regimen")["Tumor Volume (mm3)"].var()
std = remove_g989.groupby("Drug Regimen")["Tumor Volume (mm3)"].std()
stderr = remove_g989.groupby("Drug Regimen")["Tumor Volume (mm3)"].sem()

# Assemble the resulting series into a single summary DataFrame.
summary_df = pd.DataFrame({
"Mean Tumor Volume" : mean,
"Median Tumor Volume" : median,
"Tumor Volume Variance" : variance,
"Tumor Volume Std. Dev." : std,
"Tumor Std. Err" : stderr
})
summary_df

In [ ]:
# A more advanced method to generate a summary statistics table of mean, median, variance, standard deviation,
# and SEM of the tumor volume for each regimen (only one method is required in the solution)

# Using the aggregation method, produce the same summary statistics in a single line
ad_summary=pd.DataFrame(remove_g989.groupby("Drug Regimen")["Tumor Volume (mm3)"].agg(["mean", "median", "var", "std", "sem"]))
ad_summary

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using Pandas.
counts = remove_g989["Drug Regimen"].value_counts()
counts.plot(kind="bar")
counts_plot = plt.ylabel("# Observed Mouse Timepoints")
plt.show()

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using pyplot.
pandas = plt.bar(counts.index, counts)
plt.xlabel("Drug Regimen")
plt.ylabel("# Observed Mouse Timepoints")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas
sex = remove_g989.groupby(["Sex"])["Mouse ID"].count()
sex.plot(kind="pie", autopct="%1.1f%%")
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
plt.pie(sex,labels=sex.index, autopct="%1.1f%%")
plt.show()

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse
last_timepoint = remove_g989.groupby("Mouse ID")["Timepoint"].max()
last_timepoint_df = pd.DataFrame(last_timepoint, index=last_timepoint.index)

# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint
merged_df = pd.merge(last_timepoint, remove_g989, on=["Mouse ID", "Timepoint"], how="inner")
merged_df = merged_df.loc[(merged_df["Drug Regimen"] == 'Capomulin') \
                        | (merged_df["Drug Regimen"] == 'Ramicane') \
                        | (merged_df["Drug Regimen"] == 'Infubinol') \
                        | (merged_df["Drug Regimen"] =='Ceftamin') ]
merged_df.head()

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
treatment = ["Capomulin", "Ramicane", "Infubinol", "Ceftamin"]

# Create empty list to fill with tumor vol data (for plotting)
tumor_volume_final = []
    
# Locate the rows which contain mice on each drug and get the tumor volumes
# Calculate the IQR and quantitatively determine if there are any potential outliers. 
for drug in treatment:
    volume = merged_df["Tumor Volume (mm3)"][merged_df["Drug Regimen"] == drug]
    quartiles = volume.quantile([0.25, 0.5, 0.75])
    firstq = quartiles[0.25]
    thirdq = quartiles[0.75]
    iqr = thirdq - firstq

    # Determine outliers using upper and lower bounds
    outliers = []
    for vol in volume:
        if (vol > thirdq + (1.5 * iqr)) or (vol < (firstq - 1.5 * iqr)):
            outliers.append(vol)
    print(f'{drug}\'s potential outliers are {outliers}.')
    tumor_volume_final.append(tuple(volume))

In [ ]:
# Generate a box plot that shows the distribution of the tumor volume for each treatment group.
plt.boxplot(tumor_volume_final, flierprops={"markerfacecolor":"red", "markersize":12})
plt.ylabel("Final Tumor Volume (mm3)")
plt.xlabel("Treatment")
ticks = [i + 1 for i in range(len(treatment))]
plt.xticks(ticks, labels=treatment)
plt.show()

## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a single mouse treated with Capomulin
# Create df of all mice treated with Capomulin to plot
capomulin = remove_g989[remove_g989["Drug Regimen"] == 'Capomulin']

# Create df for chosen mouse
mouse_i557 = capomulin.loc[capomulin["Mouse ID"] == "i557"]

# Define dimension and metrics for i557
time = mouse_i557["Timepoint"]
tumor_volume = mouse_i557["Tumor Volume (mm3)"]

# Plot and customize line chart
plt.plot(time, tumor_volume)
plt.xlim(0, max(time))
plt.xlabel("Timepoint (Days)")
plt.ylabel("Tumor Volume (mm3)")
plt.title("Capomulin Treatment of Mouse i557")
plt.show()

In [ ]:
# Generate a scatter plot of mouse weight vs. the average observed tumor volume for the entire Capomulin regimen
# Isolate weight of mouse and calculate tumor volume of each
mouse_weight = capomulin.groupby("Mouse ID")["Weight (g)"].max()
avg_tumor_weight = capomulin.groupby("Mouse ID")["Tumor Volume (mm3)"].mean()

# Create scatterplot of avg. tumor vol by mouse weight
plt.scatter(mouse_weight, avg_tumor_weight)
plt.xlabel("Mouse Weight (g)")
plt.ylabel("Avg. Tumor Volume (mm3)")
plt.title("Avg. Tumor Volume by Mouse Weight")
plt.show()

## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and a linear regression model 
# for mouse weight and average observed tumor volume for the entire Capomulin regimen
slope, intercept, r_value, p_value, stderr = st.linregress(mouse_weight, avg_tumor_weight)
line = (slope * mouse_weight) + intercept

plt.scatter(mouse_weight, avg_tumor_weight)
plt.plot(mouse_weight, line, color = "red")
plt.xlabel("Mouse Weight (g)")
plt.ylabel("Avg. Tumor Volume (mm3)")
plt.title("Avg. Tumor Volume by Mouse Weight")
plt.show()

print(f'The correlation coefficient between mouse weight and average mouse is {round(r_value,2)}.')